# etl_silver_to_gold
---
Este notebook executa o processo `ETL` que transfere os dados da camada **Silver** para a **Gold**, englobando normalização, movimentação dos arquivos e carga dos dados no *PostgreSQL*, dando finalidade ao pipeline.


In [1]:
# Parameters

run_mode = "latest"
run_date = None

silver_path = "/opt/airflow/data-layer/silver"
gold_path = "/opt/airflow/data-layer/gold"

aggregated_name = "flights_aggregated.parquet"
postgres_conn_id = "AIRFLOW_VAR_POSTGRES_CONN_ID"


In [ ]:
import os
import shutil
from datetime import datetime
from pathlib import Path

from transformer.utils.file_io import find_partition
from transformer.utils.logger import get_logger
from transformer.utils.spark_helpers import get_spark_session, load_to_postgres, read_from_postgres
from transformer.utils.postgre_helpers import assert_table_rowcount
from transformer.utils.quality_gates_gold import run_quality_gates_gold

from pyspark.sql import DataFrame, functions as F
from pyspark.sql.types import DateType
from pyspark.sql import SparkSession


## Job 1: build_and_load_gold_star_schema

Este job realiza a construção do esquema estrela da camada **Gold**, materializando as tabelas dimensionais e fato a partir da tabela `silver_flights`, salva os dados em formato `parquet` na camada **Gold** e carregando os dados no *PostgreSQL* de acordo com o ddl da camada.


In [ ]:
log = get_logger("build_and_load_gold_star_schema")

spark = get_spark_session("BuildLoadGoldStarSchema")
log.info("[BuildLoad] SparkSession iniciada.")


### Definindo função de materialização

In [4]:
def materialize_gold_layer(df: DataFrame) -> dict[str, DataFrame]:
    """
    Materializa as tabelas dimensionais e fato da camada gold a partir do DataFrame agregado.

    Args:
        df (DataFrame): DataFrame consolidado da camada gold.

    Returns:
        dict[str, DataFrame]: DataFrames correspondentes a dim_air, dim_apt, dim_dat e fat_flt.
    """
    # Feriados federais nos EUA em 2015 (UTC)
    us_holidays_2015 = [
        "2015-01-01",
        "2015-01-19",
        "2015-02-16",
        "2015-05-25",
        "2015-07-04",
        "2015-09-07",
        "2015-10-12",
        "2015-11-11",
        "2015-11-26",
        "2015-12-25",
    ]
    holidays_df = (
        spark.createDataFrame([(d,) for d in us_holidays_2015], ["holiday_date"])
            .withColumn("holiday_date", F.col("holiday_date").cast(DateType()))
    )
    
    log.info("[Materialize] Iniciando materialização da camada gold.")

    # dim_air
    log.info("[Materialize] Materializando 'dim_air'.")
    dim_air = (
        df.select("airline_iata_code", "airline_name")
            .distinct()
            .withColumn("airline_id", F.monotonically_increasing_id())
            .select("airline_id", "airline_iata_code", "airline_name")
    )

    # dim_apt
    log.info("[Materialize] Materializando 'dim_apt'.")
    dim_apt = (
        df.select(
            F.col("origin_airport_iata_code").alias("airport_iata_code"),
            F.col("origin_airport_name").alias("airport_name"),
            F.col("origin_city").alias("city_name"),
            F.col("origin_state").alias("state_code"),
            F.col("origin_latitude").alias("latitude"),
            F.col("origin_longitude").alias("longitude")
        )
            .union(
                df.select(
                    F.col("dest_airport_iata_code").alias("airport_iata_code"),
                    F.col("dest_airport_name").alias("airport_name"),
                    F.col("dest_city").alias("city_name"),
                    F.col("dest_state").alias("state_code"),
                    F.col("dest_latitude").alias("latitude"),
                    F.col("dest_longitude").alias("longitude")
                )
            )
            .distinct()
            .withColumn("airport_id", F.monotonically_increasing_id())
            .select("airport_id", "airport_iata_code", "airport_name", "city_name",
                    "state_code", "latitude", "longitude")
    )

    # dim_dat
    log.info("[Materialize] Materializando 'dim_dat'.")
    dim_dat = (
        df.select(F.col("flight_date").alias("full_date"))
            .distinct()
            .withColumn("year", F.year("full_date"))
            .withColumn("month", F.month("full_date"))
            .withColumn("day", F.dayofmonth("full_date"))
            .withColumn("day_of_week", F.dayofweek("full_date"))
            .withColumn("quarter", F.quarter("full_date"))
            # Augmentation de feriado
            .join(holidays_df, F.col("full_date") == F.col("holiday_date"), "left")
            .withColumn("is_holiday", F.when(F.col("holiday_date").isNotNull(), F.lit(True)).otherwise(F.lit(False)))
            .drop("holiday_date")
            # Fim do augmentation de feriado
            .select("full_date", "year", "month", "day", "day_of_week", "quarter", "is_holiday")
    )

    # fat_flt
    log.info("[Materialize] Materializando 'fat_flt'.")
    fat_flt = (
        df.withColumn("flight_id", F.monotonically_increasing_id())
            .withColumnRenamed("flight_date", "full_date")
            .select(
                "flight_id", "full_date", "airline_iata_code",
                "origin_airport_iata_code", "dest_airport_iata_code",
                "distance",
                "air_time", "elapsed_time", "scheduled_time",
                "taxi_out", "taxi_in",
                "departure_delay", "arrival_delay", "air_system_delay", 
                "security_delay", "airline_delay", "late_aircraft_delay", "weather_delay"
            )
    )

    # Joins para adicionar fk's
    log.info("[Materialize] Adicionando fk's na 'fat_flt'.")
    fat_flt = (
        fat_flt
            .join(dim_air, on="airline_iata_code", how="left")
            .join(dim_apt.select(
                F.col("airport_id").alias("origin_airport_id"),
                F.col("airport_iata_code").alias("origin_airport_iata_code")
            ), on="origin_airport_iata_code", how="left")
            .join(dim_apt.select(
                F.col("airport_id").alias("dest_airport_id"),
                F.col("airport_iata_code").alias("dest_airport_iata_code")
            ), on="dest_airport_iata_code", how="left")
            .select(
                "flight_id", "full_date", "airline_id", "origin_airport_id", "dest_airport_id",
                "distance", "air_time", "elapsed_time", "scheduled_time", "taxi_out",
                "taxi_in", "departure_delay", "arrival_delay",
                "air_system_delay", "security_delay", "airline_delay",
                "late_aircraft_delay", "weather_delay"
            )
    )

    log.info("[Materialize] Materialização concluída.")
    
    return {
        "dim_air": dim_air,
        "dim_apt": dim_apt,
        "dim_dat": dim_dat,
        "fat_flt": fat_flt
    }


### Runner para o job `build_and_load_gold_star_schema`

In [ ]:
try:
    log.info("[BuildLoad] Iniciando job de materialização da gold.")

    df = read_from_postgres(
        spark=spark,
        db_conn_id=postgres_conn_id,
        table_name="silver.silver_flights",
    )

    log.info(f"[BuildLoad] Datasets carregado a partir do PostgreSQL.")

    # Materializando
    tables = materialize_gold_layer(df)
    dim_air = tables["dim_air"]
    dim_apt = tables["dim_apt"]
    dim_dat = tables["dim_dat"]
    fat_flt = tables["fat_flt"]

    # Executa quality gate
    log.info("[BuildLoad] Iniciando quality gate.")
    run_quality_gates_gold(
        dim_air=dim_air,
        dim_apt=dim_apt,
        dim_dat=dim_dat,
        fat_flt=fat_flt
    )
    log.info("[BuildLoad] Quality gate concluído com sucesso.")

    # Define partição de saída
    processing_date = datetime.now().strftime("%Y-%m-%d")
    output_dir = Path(gold_path) / processing_date / "PARQUET"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Escreve os arquivos na gold (para debug)
    log.info("[BuildLoad] Iniciando escrita dos arquivos na camada gold.")

    dim_air.write.mode("overwrite").parquet(str(output_dir / "dim_air.parquet"))
    dim_apt.write.mode("overwrite").parquet(str(output_dir / "dim_apt.parquet"))
    dim_dat.write.mode("overwrite").parquet(str(output_dir / "dim_dat.parquet"))
    fat_flt.write.mode("overwrite").parquet(str(output_dir / "fat_flt.parquet"))

    log.info("[BuildLoad] Escrita concluída com sucesso.")

    log.info("[BuildLoad] Iniciando carga da gold.")

    tables = {
        "dim_air": dim_air,
        "dim_apt": dim_apt,
        "dim_dat": dim_dat,
        "fat_flt": fat_flt,
    }

    # Carga no PostgreSQL e validação
    for table_name, df in tables.items():
        full_table_name = f"gold.{table_name}"

        log.info(f"[BuildLoad] Carregando tabela: {full_table_name}.")
        expected_count = df.count()

        # Carga no PostgreSQL
        load_to_postgres(
            df=df,
            db_conn_id=postgres_conn_id,
            table_name=full_table_name,
            mode="overwrite"
        )

        log.info(f"[BuildLoad] Tabela '{full_table_name}' carregada. Validando integridade.")

        # Validação (fallback se falhar)
        try:
            assert_table_rowcount(
                db_conn_id=postgres_conn_id,
                table_name=full_table_name,
                expected_count=expected_count,
            )
        except Exception as e:
            log.error(f"[BuildLoad][ERROR] Validação falhou para '{full_table_name}'. Limpando tabela.")

            import psycopg2

            with psycopg2.connect(
                host=os.getenv("DB_HOST", "localhost"),
                dbname=os.getenv("DB_NAME", "postgres"),
                user=os.getenv("DB_USER", "postgres"),
                password=os.getenv("DB_PASSWORD", "postgres"),
            ) as conn_pg:
                with conn_pg.cursor() as cur:
                    cur.execute(f"TRUNCATE TABLE {full_table_name} CASCADE;")
                    conn_pg.commit()

            raise ValueError(f"[BuildLoad][ERROR] Falha na validação da tabela '{full_table_name}'.") from e

        log.info(f"[BuildLoad] Validação concluída com sucesso: {full_table_name}.")

    log.info("[BuildLoad] Carga de todas as tabelas concluída com sucesso.") 

except Exception as e:
    log.exception(f"[BuildLoad][ERROR] Falha na construção do esquema estrela: {e}.")
    raise

finally:
    log.info("[BuildLoad] Job de materialização da gold encerrado.")


In [11]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

# Verifica arquivos

df_show = {
    "dim_air": dim_air,
    "dim_apt": dim_apt,
    "dim_dat": dim_dat,
    "fat_flt": fat_flt
}

for name, d in df_show.items():
    print(f"\n{name}\n")
    d.printSchema()
    d.limit(1).show(truncate=True)

# Verifica tabelas

jdbc_url = f"jdbc:postgresql://{os.getenv('DB_HOST', 'localhost')}:{os.getenv('DB_PORT', '5432')}/{os.getenv('DB_NAME', 'postgres')}"
connection_properties = {
    "user": os.getenv("DB_USER", "postgres"),
    "password": os.getenv("DB_PASSWORD", "postgres"),
    "driver": "org.postgresql.Driver",
}

tables_to_check = ["dim_air", "dim_apt", "dim_dat"]
for tbl in tables_to_check:
    print(f"\n gold.{tbl} \n")
    df_check = spark.read.jdbc(url=jdbc_url, table=f"gold.{tbl}", properties=connection_properties)
    df_check.limit(1).show(truncate=True)


In [ ]:
# Encerrando a sessão do Spark.
spark.stop()
log.info("[BuildLoad] Sessão Spark finalizada.")
